In [1]:
#not random, just number sequence
#1: request others name and record name (ask everyone?)
#2: generate its own name, publish it to let others confirm there is no conflicts
#3: one own channel is created, if no confilct, json format reading.
#3: if conflict, random a number to wait on their "own" chanel, and claim it as its own. the other start from 1.
#4: in their own channel, when other want to send some special message like raw data, a new channel is created.
#4: may have two different channel to send and recieve data for a client. 

In [3]:
from util import *

In [4]:
# used mqtt channels
# request all
# names: for request all
# name: for adding name to names

In [5]:
from collections import defaultdict,deque
import asyncio
class ControlA(ControlAdvance):    
    def pipeline_checker(self,topic):
        #decrypted
        #should let others give a callback to it then let it use caller when state updated.
        #TODO:
        #I mean, it's good to do, but writing this kind of function is a little bit mind blowing.
        return self.pipeline.state_dict[topic]
    
    
    @loop()
    async def pipeline_caller(self):
        if not self.new_message:return
        self.new_message-=1
        for key,item in self.pipeline.state_dict.items():
            if len(item):
                i=item.popleft()
                ret=self.requests_all(key,i)
                if ret is None:
                    print("request not handled")
                    item.append(i)

    def start(self):
        import asyncio
        # start
        # self.pipeline_caller()
                        
    # def update(self):
        # self.pipeline_caller()
    
    requests_func_list=list()
    def requests_all(self,topic,payload):
        for func in self.requests_func_list:
            #(C,S,t,p)
            
            # TODO: ?try more error?
            try:
                ret=func(self,self.pipeline,topic,payload)
            except AttributeError:pass
            if ret is None:continue
            return ret
    
    class Channel:
        json=False
        auto_define=True
        sended=False
        pickle=True
        def set_json(self):
            self.auto_define=False
            self.json=True
    
    sended_warned=0
    channels=defaultdict(Channel)
    new_message=0
    def on_message(self,client,userdata,msg):
        
        topic=msg.topic
        payload=msg.payload

        # print(f"Have a message that is ({topic},{payload})")
        if self.do_not_hear_itself_publish(topic,payload):return
        # print("is recieved")
        
        channel=self.channels[topic]
        
        if channel.pickle and not self.sended_warned and not channel.sended:
            print("always using pickle as protocol")
            self.sended_warned=1
        if channel.auto_define and not self.sended_warned and not channel.sended:
            print("impossible to auto_define without sending data first")
            self.sended_warned=1
            
        self.new_message+=1
        
        if channel.pickle:
            import pickle
            payload=pickle.loads(payload)
            return self.pipeline(topic,payload)    
        if channel.json:
            import json
            payload=json.loads(payload.decode())
            return self.pipeline(topic,payload)
        return self.pipeline(topic,payload)
        
    subscribed=list()
    def subscribe(self,topic:str):
        self.client.subscribe(topic)
        return self.subscribed.append(topic)
        
    pop_next_recv=list()
    def do_not_hear_itself_publish(self,topic,payload):
        #I mean, if you subscribe to a channel contians itself.
        #being able to send in that channel is hard.(without bringing a long self id)
        tu=(topic,payload)
        if not tu in self.pop_next_recv:return
        #do not go through pipeline (state_saver). handle all by itself.
        
        self.pop_next_recv.remove(tu)
        return True
        
        
        
    def publish(self,topic,payload):
        def json_transform(payload):
            import json
            return json.dumps(payload).encode()
        
        def pickle_transform(payload):
            import pickle
            return pickle.dumps(payload)
        
        channel=self.channels[topic]
        
        # if channel.auto_define and not channel.sended:
            # channel.json=not payload is bytes
            
        
        
        if channel.pickle:
            payload=pickle_transform(payload)
        elif channel.json:
            payload=json_transform(payload)
        
        if topic in self.subscribed:
            self.pop_next_recv.append((topic,payload))
        
        self.client.publish(topic,payload)
        
        channel.sended=True
        
        

In [6]:
C=ControlA()

ControlA:Connected with result code 0


In [2]:
def name_iterator(name_list,preferred_name=None):
    def name_yielder():
        if preferred_name is not None:
            yield preferred_name
        yield from range(int(10e5))
    
    for i in name_yielder():
        i=str(i)
        if i not in name_list:
            return i

In [12]:
def names(C,S,topic,payload):
    if not topic=="requests_all":return
    if not payload=="names":return
    C.publish("names",list(S.names))
    return True
    
C.requests_func_list.append(names)

In [13]:
import asyncio
print("""
    be aware that names are not updated.
    names can get very long if not having a full rest, with every client restart.
    
    """)
async def task_1(C,S):
    
    
    #1: request others name and record name (ask everyone?)
    C.subscribe("names")
    # C.channels["names"].set_json()
    C.publish("requests_all","names")
    # get_others requested name
    await asyncio.sleep(0.1)
    li=C.pipeline_checker("names")
    eli=[]
    haved_len=0
    for i in range(len(li)):
        lii=li.pop()
        eli.extend(lii)
        if haved_len!=len(lii):
            if haved_len==0:
                haved_len=len(lii)
            else:
                print("not identical name list, should be error")
                print(lii,haved_len)
                print("="*20)
    S.names=set(eli)
    return True
    

In [14]:
s=set((1,2))
s.add(3)
s,3 in s

({1, 2, 3}, True)

In [15]:
def record_names(C,S,topic,payload):
    if not topic=="name":return
    print("let's GO")
    name=payload
    if name in S.names:
        C.publish(S.name,"name already taken")
        print(name)
        if name == S.name:
            C.publish(S.name,"That Is My Name")
            print("my name run in to confict")
    else:S.names.add(name)
    return True
    
C.requests_func_list.append(record_names)


async def task_2(C,S):
    import time
    #2: generate its own name, publish it to let others confirm there is no conflicts
    C.subscribe("requests_all")
    S.name=name_iterator(S.names)
    S.name_time=time.time()
    S.names.add(S.name)
    

    await asyncio.sleep(0.1)
    
    #3: one own channel is created, for reciving data.
    C.publish(S.name,"I should be first and only.")
    C.subscribe(S.name)
    
    await asyncio.sleep(0.1)
    
    return True

In [16]:
import random
random.randint(1,10)

1

In [17]:
def respond_names(C,S,topic,payload):
    if not hasattr(S,"name"):return
    if not topic==S.name:return
    # if not topic=="name":return
    thing=payload
    if thing=="I should be first and only.":print("that is a big error")
    print("that should be my channel?")
    return True
    
C.requests_func_list.append(respond_names)

import asyncio
async def task_3(C,S):
    #3:  if confilct, random a number to wait on their "own" chanel, and claim it as its own. the other start from 1.
    import random
    for i in range(3):
        await asyncio.sleep(random.randint(1,100)*0.001)
        li=C.pipeline_checker("name")
        if not li:break
        print(li)
        print("TODOTODOTODO")
    
    else:
        print("bad_luck")
    
    if li:
        return False
    
    C.publish("name",S.name)
    C.subscribe("name")
    
    await asyncio.sleep(0.1)
    return True
    
    
    
    

In [3]:
"abc.xabc".replace("abc","",1)

'.xabc'

In [18]:

#3: if conflict, random a number to wait on their "own" chanel, and claim it as its own. the other start from 1.


In [19]:
def hihi(C,S,topic,payload):
    if not topic=="hihi":return
    return True
C.pipeline.func_list=[hihi,]

def recv_data(C,S,topic,payload):
    if not hasattr(S,"name"):return
    if not topic.startswith(S.name):return
    topic=topic.replace(S.name,"",1)
    topic=topic.replace("/","",1)
    for func in S.func_list:
        ret=func(C,S,topic,payload)
        if ret==None:continue
        return ret
    
C.requests_func_list.append(recv_data)


async def task_4(C,S):
    #3:  if no confilct, json format reading.
    C.subscribe(S.name+"/#")
    #4: in their own channel, when other want to send some special message like raw data, new channels are created.
    
    return True
    
    

In [21]:
class StateMachine:
    def __init__(self,):
        ...
    def state(self,):
        ...
    def next_step(self,):
        ...
    

In [20]:
async def await_tasks(C):
    S=C.pipeline
    first=True
    while 1:
        if not first:print("TODO~!!!,need to clear pipeline if need to restart")
        first=False
        if not await task_1(C,S):continue
        if not await task_2(C,S):continue    
        if not await task_3(C,S):continue    
        if not await task_4(C,S):continue
        break
    return True